In [ ]:
import pandas as pd
import shutil
import os


In [ ]:
data_path = './assets/data.xlsx'
if not os.path.isfile(data_path):
  from google_drive_downloader import GoogleDriveDownloader as gdd

  data_path = './data.xlsx'
  gdd.download_file_from_google_drive(file_id='1psRf7Nyg_iZPX5fkGq19RGkr6M08jKAd',
                                      dest_path=data_path,
                                      overwrite=True)

In [ ]:
## loads excel
xls = pd.ExcelFile(data_path)
training_data = pd.read_excel(xls, '학습데이터')
priority = pd.read_excel(xls, '우선순위')
data_without_ground_truth = training_data.values[:, 2]
!pip install konlpy

In [ ]:
## Kkma sentence
from konlpy.tag import Kkma
import copy
import time
kkma = Kkma()


def perform_Kkma(text):
  temp_kkma = kkma.sentences(text)
  if [text] != temp_kkma:
    new_string_to_add = ''
    new_string_to_add += ' '.join(temp_kkma)
    return new_string_to_add
  else:
    return text


s = time.time()

training_data_copy_kkma = copy.deepcopy(training_data)

cnt = 0
column_added_kkma = []
for text in training_data['발화']:
  column_added_kkma.append(perform_Kkma(text))
  cnt += 1
training_data_copy_kkma['Kkma'] = column_added_kkma

print(training_data.values[:, 2][:20])
print(training_data_copy_kkma.values[:, 10][:20])
e = time.time()

print(e - s)

In [21]:
!pip install fasttext

     |████████████████████████████████| 71kB 2.9MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3098372 sha256=cb3db6caa6fcaf98dd8bb15f36b12b44a53bbe97e728de61a465b863c5f3fb0f
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [23]:
import fasttext
help(fasttext.FastText)

Help on module fasttext.FastText in fasttext:

NAME
    fasttext.FastText

DESCRIPTION
    # Copyright (c) 2017-present, Facebook, Inc.
    # All rights reserved.
    #
    # This source code is licensed under the MIT license found in the
    # LICENSE file in the root directory of this source tree.

FUNCTIONS
    cbow(*kargs, **kwargs)
    
    eprint(*args, **kwargs)
    
    load_model(path)
        Load a model given a filepath and return a model object.
    
    read_args(arg_list, arg_dict, arg_names, default_values)
    
    skipgram(*kargs, **kwargs)
    
    supervised(*kargs, **kwargs)
    
    tokenize(text)
        Given a string of text, tokenize it and return a list of tokens
    
    train_supervised(*kargs, **kwargs)
        Train a supervised model and return a model object.
        
        input must be a filepath. The input text does not need to be tokenized
        as per the tokenize function, but it must be preprocessed and encoded
        as UTF-8. You might wan

In [24]:
!wget https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz

--2021-07-15 09:05:42--  https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 457609 (447K) [application/x-tar]
Saving to: ‘cooking.stackexchange.tar.gz’

cooking.stackexchan 100%[===================>] 446.88K   608KB/s    in 0.7s    

2021-07-15 09:05:44 (608 KB/s) - ‘cooking.stackexchange.tar.gz’ saved [457609/457609]

cooking.stackexchange.id
cooking.stackexchange.txt
readme.txt


In [26]:
!head cooking.stackexchange.txt

__label__sauce __label__cheese How much does potato starch affect a cheese sauce recipe?
__label__food-safety __label__acidity Dangerous pathogens capable of growing in acidic environments
__label__cast-iron __label__stove How do I cover up the white spots on my cast iron stove?
__label__restaurant Michelin Three Star Restaurant; but if the chef is not there
__label__knife-skills __label__dicing Without knife skills, how can I quickly and accurately dice vegetables?
__label__storage-method __label__equipment __label__bread What's the purpose of a bread box?
__label__baking __label__food-safety __label__substitutions __label__peanuts how to seperate peanut oil from roasted peanuts at home?
__label__chocolate American equivalent for British chocolate terms
__label__baking __label__oven __label__convection Fan bake vs bake
__label__sauce __label__storage-lifetime __label__acidity __label__mayonnaise Regulation and balancing of readymade packed mayonnaise and other sauces


In [27]:
!wc cooking.stackexchange.txt

  15404  169582 1401900 cooking.stackexchange.txt


In [28]:
!head -n 12404 cooking.stackexchange.txt > cooking.train

In [29]:
!tail -n 3000 cooking.stackexchange.txt > cooking.valid

In [30]:
model = fasttext.train_supervised(input="cooking.train")

In [31]:
model.predict("Which baking dish is best to bake a banana bread ?")

(('__label__baking',), array([0.07257967]))

In [32]:
model.predict("Why not put knives in the dishwasher?")

(('__label__food-safety',), array([0.07451777]))

In [33]:
model.test("cooking.valid")

(3000, 0.135, 0.05838258613233386)

In [34]:
model.test("cooking.valid", k=5)

(3000, 0.06606666666666666, 0.14285714285714285)

In [35]:
model.predict("Why not put knives in the dishwasher?", k=5)

(('__label__food-safety',
  '__label__baking',
  '__label__bread',
  '__label__substitutions',
  '__label__equipment'),
 array([0.07451777, 0.07366108, 0.04390582, 0.0373    , 0.03408055]))

In [37]:
with open('./cooking.stackexchange.txt', 'r') as f:
  lines = f.readlines()
lines

['__label__sauce __label__cheese How much does potato starch affect a cheese sauce recipe?\n',
 '__label__food-safety __label__acidity Dangerous pathogens capable of growing in acidic environments\n',
 '__label__cast-iron __label__stove How do I cover up the white spots on my cast iron stove?\n',
 '__label__restaurant Michelin Three Star Restaurant; but if the chef is not there\n',
 '__label__knife-skills __label__dicing Without knife skills, how can I quickly and accurately dice vegetables?\n',
 "__label__storage-method __label__equipment __label__bread What's the purpose of a bread box?\n",
 '__label__baking __label__food-safety __label__substitutions __label__peanuts how to seperate peanut oil from roasted peanuts at home?\n',
 '__label__chocolate American equivalent for British chocolate terms\n',
 '__label__baking __label__oven __label__convection Fan bake vs bake\n',
 '__label__sauce __label__storage-lifetime __label__acidity __label__mayonnaise Regulation and balancing of readym

In [41]:
len(lines)

15404